# 1. Samples pre-processing

This task will fetch the raw data from the remote database given a SQL query file.

## 1.1 Import libraries

In [1]:
import os
import sys
import numpy as np
import pandas as pd

sys.path.append('..')

from utils import get_csv_files, memory_usage

## 1.2 Load the data

In [2]:
DATA_DIR = '../../data'
STORAGE_DIR = '../../storage'

model = 'samples'
model_storage = os.path.join(STORAGE_DIR, model)

if not os.path.exists(model_storage):
    os.makedirs(model_storage)

In [3]:
csv_files = get_csv_files(os.path.join(DATA_DIR, 'raw', model, '*.csv'))

In [4]:
if not csv_files:
    raise FileNotFoundError('Couldn\'t find any csv files! Please make sure the filepath exists')

In [5]:
df = pd.read_csv(csv_files[0], sep=';', nrows=100)

# sample rows to list columns and dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    100 non-null    int64  
 1   device_id             100 non-null    int64  
 2   timestamp             100 non-null    object 
 3   battery_state         100 non-null    object 
 4   battery_level         100 non-null    float64
 5   timezone              100 non-null    object 
 6   country_code          100 non-null    object 
 7   charger               100 non-null    object 
 8   health                100 non-null    object 
 9   voltage               100 non-null    float64
 10  temperature           100 non-null    float64
 11  usage                 100 non-null    float64
 12  up_time               100 non-null    int64  
 13  sleep_time            100 non-null    int64  
 14  network_status        100 non-null    object 
 15  mobile_data_status    10

## 1.3 Basic data pre-processing

In [7]:
memory_usage(df) * 5000000 / 100

4000.0